In [21]:
# Auto-reload setup
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from data_processing import *
print('Successful import')

Successful import


In [23]:
folder = '../data/raw'
types = ['prices_raw', 'total_load_day_ahead_raw', 'generation_production_raw']

grouped = group_files_by_type(folder, types, extension='.csv')
dfs = load_dataframes(grouped)

# Access each DataFrame
df_prices = dfs['prices_raw']
df_load = dfs['total_load_day_ahead_raw']
df_gen = dfs['generation_production_raw']

In [24]:
df_load = [drop_unecessary_columns(df, 'Day-ahead Total Load Forecast [MW] - BZN|FR') for df in df_load] 
df_load = [rename_and_reoder_columns(df, None, ['Date', 'Load']) for df in df_load]
df_load = [setup_time(df, 'Date', '%d.%m.%Y %H:%M') for df in df_load]
df_load = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Load') for df in df_load]

df_load = concat_data(df_load)
# df_summary(df_load)
df_load.to_csv('../data/clean/exploitable/Concat_load.csv', index=False)

OUI
OUI
OUI
OUI


In [25]:
df_gen = [drop_unecessary_columns(df, ["Area","Biomass - Actual Aggregated [MW]","Energy storage - Actual Aggregated [MW]","Fossil Brown coal/Lignite - Actual Aggregated [MW]",
                                       "Fossil Coal-derived gas - Actual Aggregated [MW]","Fossil Hard coal - Actual Aggregated [MW]",
                                       "Fossil Oil - Actual Aggregated [MW]","Fossil Oil shale - Actual Aggregated [MW]","Fossil Peat - Actual Aggregated [MW]",
                                       "Geothermal - Actual Aggregated [MW]","Hydro Pumped Storage - Actual Aggregated [MW]","Hydro Pumped Storage - Actual Consumption [MW]",
                                       "Hydro Run-of-river and poundage - Actual Aggregated [MW]","Hydro Water Reservoir - Actual Aggregated [MW]","Marine - Actual Aggregated [MW]",
                                       "Other - Actual Aggregated [MW]","Other renewable - Actual Aggregated [MW]","Waste - Actual Aggregated [MW]"]) for df in df_gen] 
df_gen = [rename_and_reoder_columns(df, None, ['Date','Fossil_gas', 'Nuclear', 'Solar', 'Wind_Offshore', 'Wind_Onshore']) for df in df_gen]
df_gen = [setup_time(df, 'Date', '%d.%m.%Y %H:%M') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Nuclear') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Fossil_gas') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Solar') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Wind_Offshore') for df in df_gen]
df_gen = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Wind_Onshore') for df in df_gen]

df_gen = concat_data(df_gen)
# df_summary(df_gen)

df_gen.to_csv('../data/clean/exploitable/Concat_gen.csv', index=False)

OUI
OUI
OUI
OUI


In [26]:
df_prices = [drop_unecessary_columns(df, ['Area', 'Sequence', 'Intraday Period (CET/CEST)', 'Intraday Price (EUR/MWh)']) for df in df_prices] 
df_prices = [rename_and_reoder_columns(df, None, ['Date', 'Price']) for df in df_prices]
df_prices = [setup_time(df, 'Date', '%d/%m/%Y %H:%M:%S') for df in df_prices]
df_prices = [fill_hourly_nans_by_rolling_mean(df, 'Date', 'Price') for df in df_prices]

df_prices = concat_data(df_prices)
# df_summary(df_prices)

df_prices.to_csv('../data/clean/exploitable/Concat_price.csv', index=False)

OUI
OUI
OUI
OUI


In [27]:
df_merged = merge_df([df_prices, df_load, df_gen], 'Date', 'inner')
df_merged.to_csv('../data/clean/exploitable/merged_data.csv', index=False)
# df_summary(df_merged)